# **Inspecting and Adjusting Schema**

Schema - describes the structure of normalised data

- **Ways to Inspect Schema in `dlt`**
1. CLI
2. With Python
3. Export Schema directly

In [2]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator
import os


os.environ["SOURCES__SECRET_KEY"] = os.getenv("GITHUB_TOKEN")


@dlt.source
def github_source(secret_key=dlt.secrets.value):
    client = RESTClient(
            base_url="https://api.github.com",
            auth=BearerTokenAuth(token=secret_key),
            paginator=HeaderLinkPaginator(),
    )

    @dlt.resource
    def github_pulls(cursor_date=dlt.sources.incremental("updated_at", initial_value="2024-12-01")):
        params = {
            "since": cursor_date.last_value,
            "status": "open"
        }
        for page in client.paginate("repos/dlt-hub/dlt/pulls", params=params):
            yield page


    return github_pulls


# define new dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="lesson_7",
    destination="duckdb",
    dataset_name="github_data",
)


# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)

Pipeline lesson_7 load step completed in 0.97 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:///c:\Users\HP\OneDrive\Desktop\Data Engg\dlt\Lesson Notebooks\lesson_7.duckdb location to store data
Load package 1740514802.2347448 is LOADED and contains no failed jobs


---
### **(0) CLI**

Let's first try the CLI command `dlt pipeline -v <pipeline_name> load-package`, which is used to inspect a load package in verbose mode.

> In the context of the `dlt` library, a load package is a collection of jobs with data for particular tables. The -v flag stands for verbose, which means the command will provide more detailed output.

Specifically, this command will show the schema changes introduced in the load package for the given pipeline.

In [3]:
!dlt pipeline -v lesson_7 load-package

Attaching to pipeline lesson_7
Found pipeline lesson_7 in C:\Users\HP\.dlt\pipelines
Package 1740514802.2347448 found in C:\Users\HP\.dlt\pipelines\lesson_7\load\loaded\1740514802.2347448
The package with load id 1740514802.2347448 for schema github_source is in LOADED state. It updated schema for 9 tables. The package was LOADED at 2025-02-25 20:20:04.987432+00:00.
Jobs details:
Job: github_pulls.5492f33d4c.insert_values, table: github_pulls in completed_jobs. File type: insert_values, size: 15.8K. Started on: 2025-02-25 20:20:03.822387+00:00 and completed in 1.17 seconds.
Job: github_pulls__assignees.d45cea0b37.insert_values, table: github_pulls__assignees in completed_jobs. File type: insert_values, size: 1.0K. Started on: 2025-02-25 20:20:03.827744+00:00 and completed in 1.16 seconds.
Job: github_pulls__base__repo__topics.76ebc3b9fe.insert_values, table: github_pulls__base__repo__topics in completed_jobs. File type: insert_values, size: 3.3K. Started on: 2025-02-25 20:20:03.825773+

---
### **(1) Python**

Alternatively, we can inspect the schema object from load info with:

```python
print(load_info.load_packages[0].schema)
```

which has the following public methods and attributes:

In [4]:
# This code snippet just prints out the public methoda and attributes of the schema object in load info
all_attributes_methods = dir(load_info.load_packages[0].schema)
public_attributes_methods = [attr for attr in all_attributes_methods if not attr.startswith('_')]

print(f"{'Attribute/Method':<50} {'Type':<10}")
print("-" * 40)
for attr in public_attributes_methods:
    attr_value = getattr(load_info.load_packages[0].schema, attr)
    if callable(attr_value):
        print(f"{attr:<50} {'method':<10}")
    else:
        print(f"{attr:<50} {'attribute':<10}")

Attribute/Method                                   Type      
----------------------------------------
ENGINE_VERSION                                     attribute 
add_type_detection                                 method    
apply_schema_contract                              method    
clone                                              method    
coerce_row                                         method    
data_item_normalizer                               attribute 
data_table_names                                   method    
data_tables                                        method    
dlt_table_names                                    method    
dlt_tables                                         method    
drop_tables                                        method    
expand_schema_contract_settings                    method    
filter_row                                         method    
filter_row_with_hint                               method    
from_dict                    

In [5]:
print(load_info.load_packages[0].schema.to_pretty_json())

{
  "version": 2,
  "version_hash": "PFX9VoFpCl/AA3bY+A/mM1b/6V67woKy33y30GOZQLw=",
  "engine_version": 11,
  "name": "github_source",
  "tables": {
    "_dlt_version": {
      "columns": {
        "version": {
          "data_type": "bigint",
          "nullable": false
        },
        "engine_version": {
          "data_type": "bigint",
          "nullable": false
        },
        "inserted_at": {
          "data_type": "timestamp",
          "nullable": false
        },
        "schema_name": {
          "data_type": "text",
          "nullable": false
        },
        "version_hash": {
          "data_type": "text",
          "nullable": false
        },
        "schema": {
          "data_type": "text",
          "nullable": false
        }
      },
      "write_disposition": "skip",
      "resource": "_dlt_version",
      "description": "Created by DLT. Tracks schema updates"
    },
    "_dlt_loads": {
      "columns": {
        "load_id": {
          "data_type": "text",


---
### **(2) Exporting schema**

> Exporting the data schema directly into a file might be even more straightforward than the two previous approaches.

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="lesson_7_export",
    destination="duckdb",
    dataset_name="github_data",
    export_schema_path="schemas/export", # <--- dir path for a schema export
)

load_info = pipeline.run(github_source())
print(load_info)


Pipeline lesson_7_export load step completed in 0.92 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:///c:\Users\HP\OneDrive\Desktop\Data Engg\dlt\Lesson Notebooks\lesson_7_export.duckdb location to store data
Load package 1740515103.029457 is LOADED and contains no failed jobs


#### **Components of the YAML Schema File**

**Version hash**
- It is produced every time the schema changes
- On the first run there are already two previous hashes because creates different schemas in the extract and normalize phase as well
- version number can be used as an indicator for how many times the schema changes but not when there's parallel processing involved
- it is stored in `__dlt_version` table -> 

- On subsequent runs, `dlt` checks if the generated schema hash is stored in this table. If it is not, `dlt` concludes that the schema has changed and migrates the destination accordingly.

    - If multiple pipelines are sending data to the same dataset and there is a clash in table names, a single table with the union of the columns will be created.
    - If columns clash and have different types or other incompatible characteristics, the load may fail if the data cannot be coerced.

**Naming Convention**

Each schema contains a naming convention that is denoted in the following way when the schema is exported:

```yaml
...
normalizers:
  names: snake_case # naming convention
...
```
The naming convention is particularly useful if the identifiers of the data to be loaded (e.g., keys in JSON files) need to match the namespace of the destination (such as Redshift, which accepts case-insensitive alphanumeric identifiers with a maximum of 127 characters). This convention is used by `dlt` to translate between these identifiers and namespaces.

The standard behavior of `dlt` is to use the same naming convention for all destinations, ensuring that users always see the same tables and columns in their databases.

The default naming convention is `snake_case`:

- Removes all ASCII characters except alphanumerics and underscores.
- Adds an underscore (`_`) if the name starts with a number.
- Multiple underscores (`_`) are reduced to a single underscore.
- The parent-child relationship is expressed as a double underscore (`__`) in names.
- The identifier is shortened if it exceeds the length allowed at the destination.

> If you provide any schema elements that contain identifiers via decorators or arguments (e.g., `table_name` or `columns`), all the names used will be converted according to the naming convention when added to the schema. For example, if you execute `dlt.run(..., table_name="CamelCaseTableName")`, the data will be loaded into `camel_case_table_name`.

**Schema Settings**

The `settings` section of the schema file allows you to define various global rules that impact how tables and columns are inferred from data.

```yaml
settings:
  detections:
    ...
  default_hints:
    ...
```

**1. Detections**

You can define a set of functions that will be used to infer the data type of the column from a value. These functions are executed sequentially from top to bottom on the list.

```yaml
settings:
  detections:
    - timestamp # detects int and float values that can be interpreted as timestamps within a 5-year range and converts them
    - iso_timestamp # detects ISO 8601 strings and converts them to timestamp
    - iso_date #detects strings representing an ISO-like date (excluding timestamps) and, if so, converts to date
    - large_integer # detects integers too large for 64-bit and classifies as "wei" or converts to text if extremely large
    - hexbytes_to_text # detects HexBytes objects and converts them to text
    - wei_to_double # detects Wei values and converts them to double for aggregate non-financial reporting
```

>  `iso_timestamp` detector is enabled by default.

Detectors can be removed or added directly in code:

```python
  source = source()
  source.schema.remove_type_detection("iso_timestamp")
  source.schema.add_type_detection("timestamp")
```

**2. Column hint rules**

The `default_hints` section in the schema file is used to define global rules that apply to newly inferred columns.

> These rules are applied **after normalization**, meaning after the naming convention is applied!


By default, schema adopts column hint rules from the json(relational) normalizer to support correct hinting of columns added by the normalizer:

```yaml
settings:
  default_hints:
    foreign_key:
      - _dlt_parent_id
    not_null:
      - _dlt_id
      - _dlt_root_id
      - _dlt_parent_id
      - _dlt_list_idx
      - _dlt_load_id
    unique:
      - _dlt_id
    root_key:
      - _dlt_root_id
```


You can define column names with regular expressions as well.

```yaml
settings:
  default_hints:
  partition:
        - re:_timestamp$ #  add partition hint to all columns ending with _timestamp
```

Column hints can be added directly in code:

```python
  source = data_source()
  # this will update existing hints with the hints passed
  source.schema.merge_hints({"partition": ["re:_timestamp$"]})

```

**3. Preferred data types**

In the `preferred_types` section, you can define rules that will set the data type for newly created columns. On the left side, you specify a rule for a column name, and on the right side, you define the corresponding data type. You can use column names directly or with regular expressions to match them.

```yaml
settings:
  preferred_types:
    re:timestamp: timestamp
    inserted_at: timestamp
    created_at: timestamp
    updated_at: timestamp
```
Above, we prefer `timestamp` data type for all columns containing timestamp substring and define a exact matches for certain columns.

Preferred data types can be added directly in code as well:

```python
source = data_source()
source.schema.update_preferred_types(
  {
    "re:timestamp": "timestamp",
    "inserted_at": "timestamp",
    "created_at": "timestamp",
    "updated_at": "timestamp",
  }
)
```

## **How to Modify Schema in `dlt`**

**1. Using `dlt.resource(columns=....)`**

```python
@dlt.resource(table_name='my_table', 
              columns = {'my_column' : {
                                        'data_type':'bool', 
                                        'nullable':True
                                        }
                        })
def my_resource():
    for i in range(10):
        yield {'my_column': i % 2 == 0}
```


**2. Apply Hints**
- method in `dlt` to dynamically set or adjust aspects of your resources and pipelines
    - can be used directly in dlt.resource to specify properties such as nullability
    - When dealing with dynamically generated resources or needing to programmatically set hints, apply_hints is your tool. It's especially useful for applying hints across various collections or tables at once.
    - can be used to load your data incrementally
    - set or update the table name, columns, and other schema elements  

For example, to apply a complex data type across all collections from a MongoDB source:

```python
all_collections = ["collection1", "collection2", "collection3"]  # replace with your actual collection names
source_data = mongodb().with_resources(*all_collections)

for col in all_collections:
    source_data.resources[col].apply_hints(columns={"column_name": {"data_type": "complex"}})

pipeline = dlt.pipeline(
    pipeline_name="mongodb_pipeline",
    destination="duckdb",
    dataset_name="mongodb_data"
)
load_info = pipeline.run(source_data)
```

### **(2) Adjusting schema settings**

Maybe you've noticed, but there several ways to adjust your schema settings directly in code were already covered. This is just a recap. You can go back directly to the Schema Settings section.

Detectors can be removed or added directly in code:

```python
  source = source()
  source.schema.remove_type_detection("iso_timestamp")
  source.schema.add_type_detection("timestamp")
```

Column hints can be added directly in code:

```python
  source = data_source()
  # this will update existing hints with the hints passed
  source.schema.merge_hints({"partition": ["re:_timestamp$"]})

```

Preferred data types can be added directly in code as well:

```python
source = data_source()
source.schema.update_preferred_types(
  {
    "re:timestamp": "timestamp",
    "inserted_at": "timestamp",
    "created_at": "timestamp",
    "updated_at": "timestamp",
  }
)
```

### **(2) Importing a schema**

We mentioned that you can export a schema. In a similar fashion you can import a schema. The usual approach to use this functionaility is to export the schema first, make the adjustments and put the adjusted schema into the corresponding import folder.


The instruction to import a schema should be provided at the beginning when creating a pipeline:

```
pipeline = dlt.pipeline(
    pipeline_name="github_pipeline3",
    destination="duckdb",
    dataset_name="github_data",
    export_schema_path="schemas/export",
    import_schema_path="schemas/import",
)
```

Let's make an initial pipeline run to export schema into the file.

```
# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)
```

### **Exercise 1: Adjust import schema**

**Adjust the import schema** by adding a description of the **`github_pulls`** table.


```
github_pulls:
  columns:
    updated_at:
      incremental: true
  write_disposition: append
  resource: github_pulls
  description: Table contains all pull requests information from dlt repository
```

Run the pipeline:

load_info = pipeline.run(github_source())
print(load_info)

Check the exported schema file. It should now contain a description for the `github_pulls` table.